In [1]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

In [2]:
spark = SparkSession.builder.\
        appName("test").\
        master('local[*]').getOrCreate()
sc = spark.sparkContext

In [3]:
rdd = sc.textFile("./people.txt").map(lambda x: x.split(", ")).\
        map(lambda x: (x[0], int(x[1])))

# 自动判别字段类型

In [4]:
df = spark.createDataFrame(rdd, schema=['name', 'age'])

In [5]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [6]:
df.show(10)

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 16|
+-------+---+



In [7]:
df.createTempView('people')

In [8]:
spark.sql('SELECT * FROM people WHERE age < 30').show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
| Justin| 16|
+-------+---+



# 使用StructType构建DataFrame表

In [9]:
from pyspark.sql.types import StructType, StringType, IntegerType

In [10]:
schema = StructType().add("name", StringType(), True).\
            add("age", IntegerType(), False)

In [11]:
df = spark.createDataFrame(rdd, schema=schema)

In [12]:
df.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 16|
+-------+---+



In [13]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)



# 使用RDD的toDF方法转换

In [14]:
df = rdd.toDF(schema=schema)

In [15]:
df.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 16|
+-------+---+



# 源数据来自于Pandas的DataFrame

In [16]:
import pandas as pd

In [17]:
df_pd = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ['Jack', 'Justin', 'Ala'],
    "age": [11, 13 ,14]
})

In [18]:
df = spark.createDataFrame(df_pd, schema=['id', 'name', 'age'])

In [19]:
df.show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1|  Jack| 11|
|  2|Justin| 13|
|  3|   Ala| 14|
+---+------+---+



# 统一API从文件读取

In [20]:
schema = StructType().add("value", StringType(), True) # text文件
df = spark.read.format("text").schema(schema=schema).load("./people.txt")

In [21]:
df.show()

+-----------+
|      value|
+-----------+
|Michael, 29|
|   Andy, 30|
| Justin, 16|
+-----------+



In [22]:
df = spark.read.format("json").load("./people.json")  # json文件
df.printSchema()
df.show()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  14|   Mike|
|  18|   Mike|
+----+-------+



In [23]:
schema = StructType().add("name", StringType(), True).\
            add("age", IntegerType(), False).add("job", StringType(), False)
df = spark.read.format("csv").option("sep", ";").\
    option("header", True).option("encoding", "utf-8").\
    schema(schema).load("./people.csv")

In [24]:
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|  Manager|
|Alice|  9|Developer|
+-----+---+---------+



# DataFrame编程：DSL和SQL风格

In [25]:
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|  Manager|
|Alice|  9|Developer|
+-----+---+---------+



In [26]:
df.select(["name", "age"]).show()

+-----+---+
| name|age|
+-----+---+
|Jorge| 30|
|  Bob| 32|
|Alice|  9|
+-----+---+



In [27]:
df.filter("age < 31").show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|Alice|  9|Developer|
+-----+---+---------+



In [28]:
df.filter(df['age'] < 31).show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|Alice|  9|Developer|
+-----+---+---------+



In [29]:
df.groupBy(df['name']).count().show() # groupBy的返回值是GroupData类型
# GroupData类型：有分组关系的数据结构
# 接上聚合函数API：sum, avg, count, min, max
# 通过聚合函数后返回的是DataFrame

+-----+-----+
| name|count|
+-----+-----+
|  Bob|    1|
|Jorge|    1|
|Alice|    1|
+-----+-----+



# word count demo

In [31]:
from pyspark.sql import functions as F

In [32]:
rdd = sc.textFile('./words.txt').flatMap(lambda x: x.split(" ")).map(lambda x: [x])

In [33]:
df = rdd.toDF(["word"])

In [35]:
df.createTempView("words") # SQL风格

In [37]:
spark.sql("SELECT word, COUNT(*) AS cnt FROM words GROUP BY word ORDER BY cnt DESC").show()

+------+---+
|  word|cnt|
+------+---+
| hello|  3|
| flink|  1|
| spark|  1|
|hadoop|  1|
+------+---+



In [38]:
# DSL风格
df = spark.read.format("text").load("./words.txt")

In [43]:
# withColumn方法
df2 = df.withColumn("value", F.explode(F.split(df['value'], " ")))

In [49]:
df2.groupBy("value").count().withColumnRenamed("value", "word").\
    orderBy("count", ascending=False).show()

+------+-----+
|  word|count|
+------+-----+
| hello|    3|
| flink|    1|
| spark|    1|
|hadoop|    1|
+------+-----+



# 电影评分案例

In [89]:
schema=StructType().add("user_id", IntegerType(), False).add("movie_id", IntegerType(), False).\
    add("rank", IntegerType(), False).add("ts", StringType(), False)
df = spark.read.format("csv").schema(schema=schema).option("header", False).\
    option("encoding", "utf-8").option("sep", "\t").load("./u.data")

In [90]:
# TODO:用户的平均分
df.groupBy("user_id").avg("rank").withColumnRenamed("avg(rank)", "avg_rank").\
        withColumn("avg_rank", F.round("avg_rank", 2)).orderBy("avg_rank",ascending=False).show(5)

+-------+--------+
|user_id|avg_rank|
+-------+--------+
|    849|    4.87|
|    688|    4.83|
|    507|    4.72|
|    628|     4.7|
|    928|    4.69|
+-------+--------+
only showing top 5 rows



In [94]:
# TODO:电影的平均分查询,采用SQL风格 --> SQL一步到位，需要很熟悉
df.createOrReplaceTempView("movie")
spark.sql("SELECT movie_id, ROUND(AVG(rank), 2) AS avg_rank FROM movie GROUP BY movie_id ORDER BY avg_rank DESC").show(4)

+--------+--------+
|movie_id|avg_rank|
+--------+--------+
|    1122|     5.0|
|    1500|     5.0|
|    1201|     5.0|
|    1653|     5.0|
+--------+--------+
only showing top 4 rows



In [98]:
# TODO:查询大于电影平均分的数量
df.where(df['rank'] > df.select(F.avg(df['rank'])).first()['avg(rank)']).count()

55375

In [102]:
# TODO：查询大于3分电影中，打分次数最多的用户，此人打分的平均分
df.where(df["rank"]>3).groupBy("user_id").count().orderBy("count", ascending=False).first()['user_id']

450

In [106]:
# 计算这个人的打分平均分
df.filter(df['user_id']==450).select(F.round(F.avg("rank"), 2)).show()

+-------------------+
|round(avg(rank), 2)|
+-------------------+
|               3.86|
+-------------------+



In [109]:
# TODO:查询每个用户的平均打分，最低打分，最高打分
df.groupBy("user_id").\
    agg(
        F.round(F.avg("rank"), 2).alias("avg_rank"),
        F.min("rank").alias("min_rank"),
        F.max("rank").alias("max_rank")
    ).show(4)

+-------+--------+--------+--------+
|user_id|avg_rank|min_rank|max_rank|
+-------+--------+--------+--------+
|    148|     4.0|       1|       5|
|    463|    2.86|       1|       5|
|    471|    3.39|       1|       5|
|    496|    3.03|       1|       5|
+-------+--------+--------+--------+
only showing top 4 rows



In [113]:
# TODO:查询评分超过100次的电影的平均分
df.groupBy("movie_id").\
    agg(
        F.count("movie_id").alias("cnt"),
        F.round(F.avg("rank"), 2).alias("avg_rank")
    ).where("cnt > 100").orderBy("avg_rank", ascending=False).\
    show(5)

+--------+---+--------+
|movie_id|cnt|avg_rank|
+--------+---+--------+
|     408|112|    4.49|
|     169|118|    4.47|
|     318|298|    4.47|
|     483|243|    4.46|
|      64|283|    4.45|
+--------+---+--------+
only showing top 5 rows



In [ ]:
"""
1、agg函数：它是GroupData对象的API，作用是可以在里面写多个聚合；
2、alias：它是Column对象的API可以对一个列进行改名；
3、withColumnRenamed：它是DataFrame的API，可以对DF中的列进行改名；与alias不同
4、orderBy：DataFrame的API，对DF进行排序；
5、first：DataFrame的API，取出第一行，返回Row对象；
# Row对象可通过row['列名']来取出当前行中，某一列的具体数值；
"""

In [ ]:
# 统一API数据写出
